<a href="https://colab.research.google.com/github/stebechoi/CP2/blob/Hwkdir/3.%20(MovieLense100K)_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MovieLense 데이터를 이용한 영화 추천 시스템(Collaborative filtering: 협업필터링)

## CF
- Collaborative Filtering 협업 필터링 </br>
  사용자와 비슷한 다른 사용자의 평가를 파악하여 이용
    - 장점: 아이템 특성에 의존하지 않는다.
    - 단점
      - 평가되지 않은 아이템은 추천하지 않는다(new item problem)
      - 보통 가장 인기 있는 아이템이 추천된다.
      - 비슷한 유저 군이 존재하는 사용자 그룹이 필요하다.(cold start problem for new users)

In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### CF 유사도 모델
참고
- https://www.youtube.com/watch?v=mJ8pEjN3FKg </br>
- https://www.youtube.com/watch?v=Lc5mfCF0mCU

In [18]:
columns=['user_id', 'movie_id', 'rating', 'timestamp'] 
data=pd.read_csv('ml-100k/u.data', sep='\t', names=columns)
data = data[['user_id', 'movie_id', 'rating']]
data

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


In [19]:
# 협업필터링을 이용한 영화 추천 엔진

n_users = data['user_id'].unique().shape[0] #943
n_items = data['movie_id'].nunique() #1682

#(943,1682)형태의 0으로 이루어진 행렬
data_matrix = np.zeros((n_users, n_items)) 

# sparse matrix 형태로 구성, 위 zeros행렬에 각 값 대치
for row in data.itertuples():
  data_matrix[row[1]-1, row[2]-1]=row[3]
# itertuple이용 행에 접근할 수 있는 iterator 생성
# -1은 인덱스와 다르기 때문
# row[1]-1: 사용자정보, row[2]-1: 영화정보, row[3]:평점

# 훈련 데이터와 텍스트 데이터의 분리
from sklearn.model_selection import train_test_split

train, test = train_test_split(data_matrix, test_size=0.2, random_state=42)

# 사용자 간 유사도 행렬 생성
from sklearn.metrics.pairwise import cosine_distances

cosine_distances(train)

# 코사인 유사도: 내적 공간의 두 벡터의 사이각에 대한 코사인을 측정해 유사도를 나타내는 척도
# cos(0) == 1, cos(θ) < 1
# cosine distance함수로 벡터만 주면 바로 계산 가능
distances = 1 - cosine_distances(train) # 자기자신을 1로 하도록 변경

# 평가 예측 및 모델 성능 측정
user_pred = distances.dot(train) / np.array([np.abs(distances).sum(axis=1)]).T
# np.dot(). 행의 크기가 같아야 연산 가능
# 

from sklearn.metrics import mean_squared_error
def get_mse(pred, actual):
  pred = pred[actual.nonzero()].flatten()
  actual = actual[actual.nonzero()].flatten()
  return mean_squared_error(pred, actual)

# 훈련데이터 RMSE
a = np.sqrt(get_mse(user_pred, train)) 
# 테스트 RMSE
b = np.sqrt(get_mse(user_pred, test)) 

print(a,b)

2.799242657919795 2.989599866496873


In [20]:
data_matrix

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])